<h2>Data Dictionary</h2>
<table><tr>
    <th>Variable</th>
    <th>Definition</th>
    <th>Key</th>
    </tr><tr>
    <th>survival</th>
    <td>Survival</td>
    <td>0 = No, 1 = Yes</td>
    </tr><tr>
    <th>pclass</th>
    <td>Ticket class</td>
    <td>1 = 1st, 2 = 2nd, 3 = 3rd</td>
    </tr><tr>
    <th>sex</th>
    <td>gender</td>
    </tr><tr>
    <th>Age</th>
    <td>Age in years</td>
    </tr><tr>
    <th>sibsp</th>
    <td># of siblings / spouses aboard the Titanic</td>
    </tr><tr>
    <th>parch</th>
    <td># of parents / children aboard the Titanic</td>
    </tr><tr>
    <th>ticket</th>
    <td>Ticket number</td>
    </tr><tr>
    <th>fare</th>
    <td>Passenger fare</td>
    </tr><tr>
    <th>cabin</th>
    <td>Cabin number</td>
    </tr><tr>
    <th>embarked</th>
    <td>Port of Embarkation</td>
    <td>C = Cherbourg, Q = Queenstown, S = Southampton</td>
    </tr>
</table>
<h2>Variable Notes</h2>

<b>pclass</b>: A proxy for socio-economic status (SES)<br>
1st = Upper<br>
2nd = Middle<br>
3rd = Lower<br>

<b>age</b>: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5
<br>

<b>sibsp</b>: The dataset defines family relations in this way...
<br>Sibling = brother, sister, stepbrother, stepsister
<br>Spouse = husband, wife (mistresses and fiancés were ignored)
<br>

<b>parch</b>: The dataset defines family relations in this way...
<br>Parent = mother, father
<br>Child = daughter, son, stepdaughter, stepson
<br>Some children travelled only with a nanny, therefore parch=0 for them.

In [668]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [669]:
raw_data=pd.read_csv('train.csv')

In [670]:
raw_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [671]:
raw_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [672]:
raw_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [673]:
raw_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [674]:
for col_name in raw_data.columns:
    print(raw_data[col_name].value_counts())
    print("There are ",raw_data[col_name].isna().sum()," missing values")# age(median),cabin(droped),embarked (mode/most frequent)
    print('-'*50)

891    1
293    1
304    1
303    1
302    1
      ..
591    1
590    1
589    1
588    1
1      1
Name: PassengerId, Length: 891, dtype: int64
There are  0  missing values
--------------------------------------------------
0    549
1    342
Name: Survived, dtype: int64
There are  0  missing values
--------------------------------------------------
3    491
1    216
2    184
Name: Pclass, dtype: int64
There are  0  missing values
--------------------------------------------------
Ryerson, Miss. Susan Parker "Suzette"                1
Mallet, Mr. Albert                                   1
Nankoff, Mr. Minko                                   1
Silvey, Mrs. William Baird (Alice Munger)            1
Kelly, Miss. Anna Katherine "Annie Kate"             1
                                                    ..
Corn, Mr. Harry                                      1
Slocovski, Mr. Selman Francis                        1
Thorneycroft, Mrs. Percival (Florence Kate White)    1
Yousseff, Mr. Geriou

In [675]:
features_data=raw_data.drop(['PassengerId','Survived','Name','Ticket','Cabin'],axis=1)

In [676]:
label_data=raw_data['Survived']

In [677]:
features_data.Sex=raw_data.Sex.replace(['male','female'],[0,1])

In [678]:
age_imputer=SimpleImputer(missing_values=np.nan,strategy='median')
age_imputer

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='median', verbose=0)

In [679]:
age_imputer.fit(raw_data[['Age']])
age_imputer

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='median', verbose=0)

In [680]:
features_data.Age=age_imputer.transform(raw_data[['Age']])
features_data.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,0.352413,29.361582,0.523008,0.381594,32.204208
std,0.836071,0.477990,13.019697,1.102743,0.806057,49.693429
min,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,22.000000,0.000000,0.000000,7.910400
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200
75%,3.000000,1.000000,35.000000,1.000000,0.000000,31.000000
max,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200


In [681]:
embarked_imputer=SimpleImputer(missing_values=np.nan,strategy='most_frequent')
embarked_imputer

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='most_frequent', verbose=0)

In [682]:
embarked_imputer.fit(raw_data[['Embarked']])
embarked_imputer

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='most_frequent', verbose=0)

In [683]:
features_data.Embarked=embarked_imputer.transform(raw_data[['Embarked']])

In [684]:
features_data.Embarked=features_data.Embarked.replace(['S','C','Q'],[0,1,2])
features_data.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,0.352413,29.361582,0.523008,0.381594,32.204208,0.361392
std,0.836071,0.477990,13.019697,1.102743,0.806057,49.693429,0.635673
min,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,22.000000,0.000000,0.000000,7.910400,0.000000
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,0.000000
75%,3.000000,1.000000,35.000000,1.000000,0.000000,31.000000,1.000000
max,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,2.000000


In [685]:
features_data=features_data.drop(['Age'],axis=1)
#this boosted the average accuracy from 75% to 80%
#droping Embarked feature resulted in slight decrease in accuracy score

In [686]:
DTC_model=DTC()
DTC_model

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [687]:
train_features,test_features,train_label,test_label=train_test_split(features_data,label_data,test_size=0.25,random_state=42)
print(train_features.shape,test_features.shape,train_label.shape,test_label.shape)

(668, 6) (223, 6) (668,) (223,)


In [688]:
DTC_model.fit(train_features,train_label)
DTC_model

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [689]:
pred_label=DTC_model.predict(test_features)
pred_label

array([1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0], dtype=int64)

In [690]:
print(accuracy_score(test_label,pred_label))

0.8116591928251121
